In [73]:
# predictor
import pandas_datareader.data as web
from datetime import datetime, date
import pandas as pd
import numpy as np

class Options:
    def __init__(self, ticker):
        self.ticker = ticker
    
    def get_historical_stock_data(self, start, end):
        stocks = web.DataReader(self.ticker, 'iex', start, end)
        return stocks


    def make_stock_data_array(self, stocks):
        year_array = np.array([2016, 2017, 2018, 2019])
        month_array = np.array(range(1,13))
        day_array = np.array(range(1, 32))
        full_stock_data = np.array([])
        for year in year_array:
            for month in month_array:
                for day in day_array:
                    try:
                        current_date = date(year, month, day)
                        if current_date > date.today():
                            return full_stock_data
                        day_data = stocks.loc[str(current_date), "low"]
                        full_stock_data = np.append(full_stock_data, day_data)
                    except:
                        continue
                        
    def create_array_of_arrays(self, full_stock_data):
        feature_data = []
        labels = []
        while len(full_stock_data)>=26:
            feature_data.append(full_stock_data[:20])
            last_price = full_stock_data[19]
            max_price = max(full_stock_data[20:26])
            if max_price / last_price > 1.01:
                labels.append(1)
            else:
                labels.append(0)
            full_stock_data = full_stock_data[1:]
        
        if len(labels) == len(feature_data):
            return [labels, feature_data]
        else:
            return None        
        
    def write_to_files(self, labels, feature_data):
        labels_df = pd.DataFrame([labels])
        feature_data_df = pd.DataFrame(feature_data)
        status = False
        
        if labels_df is not None and feature_data_df is not None:
            labels_df.to_csv(index=False, path_or_buf='labels.csv', sep = ',', header = False)
            feature_data_df.to_csv(index=False, path_or_buf='feature_data.csv', sep = ',', header = False)
            status = True
        return status
        
     
if __name__ == "__main__": 
    options_data = Options('FIT')
    
    stocks = options_data.get_historical_stock_data(datetime(2016,1,1), datetime(2019,5,17))

    full_stock_data = options_data.make_stock_data_array(stocks)
    [labels, feature_data] = options_data.create_array_of_arrays(full_stock_data)

    status = options_data.write_to_files(labels, feature_data)
    if(status):
        print("SUCCESS")
    else:
        print("FAILURE")


SUCCESS
